In [2]:
import dpkt
import time
import socket
import os
import pandas as pd
import numpy as np
from IPython.display import display

### constants
g_dataDir = '../../iot_data/'
g_dataExtension = '.pcap'
g_testfile = '../../iot_data/16-09-27.pcap'
columns = ['local ip', 'local port', 'remote ip', 'remote port', 'direction', 'tcp data len', 'tcp seq num']

### initialization
devices = {}
start = time.time()
dataFiles = []
for fileName in os.listdir(g_dataDir):
    if fileName.endswith(g_dataExtension):
        dataFiles.append(os.path.join(g_dataDir, fileName))

### parse pcap files
for fileName in dataFiles:
    file = open(fileName,'rb')
    for ts, pkt in dpkt.pcapng.Reader(file):
        eth = dpkt.ethernet.Ethernet(pkt)
        if eth.type != dpkt.ethernet.ETH_TYPE_IP:
            continue

        ip = eth.data
        if ip.p != dpkt.ip.IP_PROTO_TCP:
            continue

        # only tcp traffic is analyzed
        tcp = ip.data
        srcIp = socket.inet_ntoa(ip.src)
        dstIp = socket.inet_ntoa(ip.dst)
        if srcIp.startswith('192.168.'):
            localIp = srcIp
            localPort = tcp.sport
            remoteIp = dstIp
            remotePort = tcp.dport
            direction = 'out'
        else:
            localIp = dstIp
            localPort = tcp.dport
            remoteIp = srcIp
            remotePort = tcp.sport
            direction = 'in'

        # check if dataframe is created for this ip
        if localIp not in devices:
            devices[localIp] = pd.DataFrame(columns = columns)

        # save data to corresponding device's dataframe
        df = devices[localIp]
        df.loc[df.shape[0]] = [
            localIp, localPort, remoteIp, remotePort, 
            direction, len(tcp.data), tcp.seq
        ]
        if df.shape[0] >= 100:
            break

print ("Time elapsed: " + str(time.time() - start))
for key, value in devices.items():
    display(key)

Time elapsed: 2.288846015930176


'192.168.1.106'

'192.168.1.236'

'192.168.1.166'

'192.168.1.196'

...

'192.168.1.143'

'192.168.1.120'

'192.168.1.156'

'192.168.1.239'

'192.168.1.227'